<a href="https://colab.research.google.com/github/hongting666/-/blob/main/kelly_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
r"""
Kelly Model · Institutional Edition (Fixed Parameters + Excel Report)
--------------------------------------------------------------------
Scope & Constraints
1) Asset universe (strict list):
   - Index futures/ETFs: US_SPX(ES=F/^GSPC/SPY), US_NASDAQ(NQ=F/^IXIC/QQQ), SPY, QQQ, FEZ, EWJ, FXI, GLD, SLV
   - Large-cap US equities: AAPL, TSLA, MAR, BABA, FSLR, SBUX, PLTR, NVDA
   - Hong Kong equities: 700HK(0700.HK/TCEHY), 27HK(0027.HK), 883HK(0883.HK/CEO)

2) Data source: yfinance. To switch to Bloomberg in the future, only replace `load_prices_robust()`.

3) Dashboard (exported as Excel sheets):
   - Using the latest trading day as the anchor, look back 5/10/20 years to compute range highs/lows.
   - Under from_low / from_high, when thresholds {2%, 5%, 8%} are triggered,
     hold for EVENT_EVAL_HOLD_D days; report win rate / mean / variance / Kelly_raw.

4) Trading / Weights sheets:
   - Whether to open/add today and suggested weights (based on Kelly_raw, TopK normalization, and vol targeting).
   - Rebalance schedule & performance for a given period (default: last year).
   - Full-history rebalance schedule.

5) Performance bar: CAGR ≥ 10% and R_over_MDD ≥ 3. MaxDD definition:
   - For each trade, record the deepest drawdown from ENTRY price (simple %), booked on the exit day;
     at the portfolio level, MaxDD is the minimum across all booked trades (usually negative).

6) Stop-loss: Hard stop fixed at 1.5% (ENTRY_DD_STOP_PCT=0.015), computed as a simple % from entry.

7) Output: One Excel file (default: kelly_institutional_report.xlsx).


"""

import math, warnings, re, sys, itertools, random
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional

warnings.filterwarnings("ignore")

# ==== GPU dependency check (RAPIDS: CuPy / cuDF) ====
try:
    import cupy as cp
    import cudf
    assert cp.cuda.runtime.getDeviceCount() > 0, "No NVIDIA GPU detected"
except Exception as e:
    raise RuntimeError(
        "An NVIDIA GPU with CuPy/cuDF (RAPIDS) is required and must be available.\nDetails: %s" % e
    )

import numpy as np
import pandas as pd
import yfinance as yf

# =========================
# Fixed parameters (incl. 1.5% hard stop)
# =========================
PREFERRED_MODE = "futures_first"         # options: "futures_first" / "etf_first"
ALLOW_FALLBACK_TO_ETF = True

CANONICAL_SYMBOLS_CORE = [
    "US_SPX","US_NASDAQ",
    "SPY","QQQ","FEZ","EWJ","FXI","GLD","SLV",
]
CANONICAL_SYMBOLS_FULL = CANONICAL_SYMBOLS_CORE + [
    "AAPL","TSLA","MAR","BABA","FSLR","SBUX","PLTR","NVDA",
    "700HK","27HK","883HK",
]

START = "2005-01-01"
END   = None
TRADING_DAYS = 252
MIN_ROWS_OK = 1000

# ===== Strategy parameters =====
TRADE_LAG_D = 1  # T0_OPEN (signal today, execute next day)
LOOKBACK_YEARS = 5
DEFAULT_BAND_THRESH = 0.06
BAND_THRESH: Dict[str, float] = {
    "AAPL":0.08,"TSLA":0.09,"MAR":0.08,"BABA":0.09,"FSLR":0.09,"SBUX":0.08,"PLTR":0.09,"NVDA":0.08,
    "0700.HK":0.09,"0027.HK":0.09,"0883.HK":0.09,
}

EVENT_EVAL_HOLD_D = 20
MIN_EVENTS = 30
FACTOR_GATE_MODE = "all"
FACTOR_MIN_EVENT_SHARPE = 0.35
FACTOR_MIN_WINRATE      = 0.58
FACTOR_MIN_KELLY_RAW    = 0.09

LONG_TREND_FILTER_ON = True
MA_LONG = 200
MA_SLOPE_FILTER_ON = True
MA_SLOPE_WIN = 20

# Capital management (baseline)
KELLY_FRACTION = 0.2
KELLY_MIN_USED = 0.0
CAP_PER_ASSET_LONG = 0.5
TOPK_PER_DAY = 3

# Stops / take-profit / time stop
STOP_SIGMA = 1.8
TAKE_SIGMA = 3.4
TIME_STOP_D = 120

# Hard stop (simple % from entry price)
ENTRY_DD_STOP_ON = True
ENTRY_DD_STOP_PCT = 0.015   # 1.5%

# Portfolio vol targeting & risk controls
TARGET_DAILY_VOL = 0.012
VOL_TARGET_ON  = True
VOL_REBAL_FREQ = "W"
VOL_LOOKBACK_D = 120
GROSS_LEV_MAX  = 1.0

# Drawdown brake thresholds
DRAWDOWN_BRAKE_ON = True
BRAKE_WINDOW_D = 84
BRAKE_THRESH   = 0.08
BRAKE_SCALE    = 0.30

COOLDOWN_SWEEP = [0]  # fixed cooldown=0 (baseline)

# Costs (baseline assumptions; tweak per market if needed)
GLOBAL_DEFAULT = dict(SLIPPAGE_BPS=4.0, COMMISSION_BPS=1.0, STAMP_DUTY_BPS=0.0)
COST_TABLE = {
    "ES=F": dict(SLIPPAGE_BPS=3.0, COMMISSION_BPS=0.8),
    "NQ=F": dict(SLIPPAGE_BPS=3.0, COMMISSION_BPS=0.8),
    "HSI=F":dict(SLIPPAGE_BPS=6.0, COMMISSION_BPS=1.0),
    "NK=F": dict(SLIPPAGE_BPS=4.0, COMMISSION_BPS=1.0),
    "SPY": dict(SLIPPAGE_BPS=5.0, COMMISSION_BPS=1.2),
    "QQQ": dict(SLIPPAGE_BPS=5.0, COMMISSION_BPS=1.2),
    "FEZ": dict(SLIPPAGE_BPS=6.0, COMMISSION_BPS=1.5),
    "EWJ": dict(SLIPPAGE_BPS=6.0, COMMISSION_BPS=1.5),
    "FXI": dict(SLIPPAGE_BPS=7.0, COMMISSION_BPS=1.5),
    "GLD": dict(SLIPPAGE_BPS=5.0, COMMISSION_BPS=1.2),
    "SLV": dict(SLIPPAGE_BPS=6.0, COMMISSION_BPS=1.2),
    "AAPL": dict(SLIPPAGE_BPS=8.0,  COMMISSION_BPS=1.2),
    "TSLA": dict(SLIPPAGE_BPS=10.0, COMMISSION_BPS=1.2),
    "MAR":  dict(SLIPPAGE_BPS=9.0,  COMMISSION_BPS=1.2),
    "BABA": dict(SLIPPAGE_BPS=10.0, COMMISSION_BPS=1.2),
    "FSLR": dict(SLIPPAGE_BPS=10.0, COMMISSION_BPS=1.5),
    "SBUX": dict(SLIPPAGE_BPS=9.0,  COMMISSION_BPS=1.2),
    "PLTR": dict(SLIPPAGE_BPS=11.0, COMMISSION_BPS=1.2),
    "NVDA": dict(SLIPPAGE_BPS=8.0,  COMMISSION_BPS=1.2),
    "0700.HK": dict(SLIPPAGE_BPS=12.0, COMMISSION_BPS=2.5, STAMP_DUTY_BPS=50.0),
    "0027.HK": dict(SLIPPAGE_BPS=14.0, COMMISSION_BPS=2.5, STAMP_DUTY_BPS=50.0),
    "0883.HK": dict(SLIPPAGE_BPS=14.0, COMMISSION_BPS=2.5, STAMP_DUTY_BPS=50.0),
}

SEED = 42
np.random.seed(SEED); cp.random.seed(SEED); random.seed(SEED)

# =========================
# 1) Data layer: symbol aliases, fallback download, and timezone handling
# =========================
ALIASES_FUTURES_FIRST: Dict[str, List[str]] = {
    "US_SPX":     ["ES=F", "^GSPC", "SPY"],
    "US_NASDAQ":  ["NQ=F", "^IXIC", "QQQ"],
    "HK_HSI":     ["HSI=F", "^HSI", "2800.HK"],
    "JP_NIKKEI":  ["NK=F", "^N225", "EWJ"],
    "JP_TOPIX":   ["^TOPX", "1306.T", "1550.T", "EWJ"],
    "SPY": ["SPY"], "QQQ": ["QQQ"], "FEZ": ["FEZ"], "EWJ": ["EWJ"],
    "FXI": ["FXI"], "GLD": ["GLD"], "SLV": ["SLV"],
    "AAPL":["AAPL"], "TSLA":["TSLA"], "MAR":["MAR"], "BABA":["BABA"],
    "FSLR":["FSLR"], "SBUX":["SBUX"], "PLTR":["PLTR"], "NVDA":["NVDA"],
    "700HK": ["0700.HK", "TCEHY"], "27HK":["0027.HK"], "883HK":["0883.HK","CEO"],
}
ALIASES_ETF_FIRST: Dict[str, List[str]] = {
    "US_SPX":     ["^GSPC", "SPY", "ES=F"],
    "US_NASDAQ":  ["^IXIC", "QQQ", "NQ=F"],
    "HK_HSI":     ["^HSI", "2800.HK", "HSI=F"],
    "JP_NIKKEI":  ["^N225", "EWJ", "NK=F"],
    "JP_TOPIX":   ["^TOPX", "1306.T", "1550.T"],
    "SPY":["SPY"],"QQQ":["QQQ"],"FEZ":["FEZ"],"EWJ":["EWJ"],"FXI":["FXI"],"GLD":["GLD"],"SLV":["SLV"],
    "AAPL":["AAPL"],"TSLA":["TSLA"],"MAR":["MAR"],"BABA":["BABA"],"FSLR":["FSLR"],"SBUX":["SBUX"],"PLTR":["PLTR"],"NVDA":["NVDA"],
    "700HK":["0700.HK","TCEHY"],"27HK":["0027.HK"],"883HK":["0883.HK","CEO"],
}

_def_al = lambda: ALIASES_ETF_FIRST if PREFERRED_MODE == "etf_first" else ALIASES_FUTURES_FIRST

def normalize_hk_usercode(sym: str) -> str:
    """Convert '700HK' to standard '0700.HK' format."""
    m = re.fullmatch(r"(\d{1,5})HK", sym.upper())
    if m: return f"{m.group(1).zfill(4)}.HK"
    return sym

def all_aliases_for(canonical: str) -> List[str]:
    """Given a canonical symbol, return the prioritized alias list (with fallbacks)."""
    aliases_map = _def_al()
    first = normalize_hk_usercode(canonical)
    out, seen = [], set()
    if first not in seen: seen.add(first); out.append(first)
    if canonical in aliases_map:
        for a in aliases_map[canonical]:
            if not ALLOW_FALLBACK_TO_ETF and canonical in ["HK_HSI","JP_NIKKEI","JP_TOPIX","US_SPX","US_NASDAQ"]:
                if "=F" not in a: continue
            if a not in seen: seen.add(a); out.append(a)
    if first in aliases_map:
        for a in aliases_map[first]:
            if not ALLOW_FALLBACK_TO_ETF and first in ["HK_HSI","JP_NIKKEI","JP_TOPIX","US_SPX","US_NASDAQ"]:
                if "=F" not in a: continue
            if a not in seen: seen.add(a); out.append(a)
    return out

def _try_download(sym: str, start: str, end: Optional[str]) -> Optional[pd.Series]:
    """Attempt to download adjusted close via yfinance (with timezone removal)."""
    try:
        df = yf.download(sym, start=start, end=end, auto_adjust=True, progress=False, threads=False)
        if df is not None and not df.empty and "Close" in df:
            s = df["Close"].rename(sym)
            if isinstance(s.index, pd.DatetimeIndex) and s.index.tz is not None:
                s.index = s.index.tz_localize(None)
            return s
    except Exception as e:
        print(f"[warn] yf.download failed for {sym}: {e}", file=sys.stderr)
    try:
        tk = yf.Ticker(sym)
        hist = tk.history(start=start, end=end, auto_adjust=True)
        if hist is not None and not hist.empty and "Close" in hist:
            s = hist["Close"].rename(sym)
            if isinstance(s.index, pd.DatetimeIndex) and s.index.tz is not None:
                s.index = s.index.tz_localize(None)
            return s
    except Exception as e:
        print(f"[warn] ticker.history failed for {sym}: {e}", file=sys.stderr)
    return None

def fetch_one_symbol_with_fallback(canonical: str, start: str, end: Optional[str]):
    """Download with alias fallbacks until the minimum row threshold is met."""
    tried = []
    for alias in all_aliases_for(canonical):
        tried.append(alias)
        s = _try_download(alias, start, end)
        if s is None:
            print(f"[try] {canonical} → {alias} ... no data", file=sys.stderr); continue
        if len(s) < MIN_ROWS_OK:
            print(f"[try] {canonical} → {alias} ... too few rows ({len(s)}), keep trying", file=sys.stderr); continue
        print(f"[ok] {canonical} → {alias} ({len(s)} rows)")
        return alias, s
    print(f"[fail] {canonical}: tried {tried}", file=sys.stderr)
    return None, None

def ensure_tz_naive_index(df: pd.DataFrame) -> pd.DataFrame:
    """Ensure tz-naive DatetimeIndex."""
    if isinstance(df.index, pd.DatetimeIndex) and df.index.tz is not None:
        new_df = df.copy(); new_df.index = new_df.index.tz_localize(None); return new_df
    return df

def load_prices_robust(canon_symbols: List[str], start: str, end: Optional[str]) -> Tuple[pd.DataFrame, Dict[str,str]]:
    """Batch load prices; return DataFrame and the alias map actually used."""
    frames = []; used_map: Dict[str,str] = {}
    for c in canon_symbols:
        alias, s = fetch_one_symbol_with_fallback(c, start, end)
        if s is not None:
            frames.append(s.to_frame()); used_map[c] = alias
    if not frames:
        raise RuntimeError("Failed to fetch data for all symbols. Check network connectivity and alias mapping.")
    prices = pd.concat(frames, axis=1).sort_index().ffill().dropna(how="all")
    prices = ensure_tz_naive_index(prices); prices.columns = [c.upper() for c in prices.columns]
    return prices, used_map

# =========================
# 2) GPU utilities & event statistics
# =========================
def annualize_vol(daily_vol: float) -> float:
    return daily_vol * math.sqrt(TRADING_DAYS)

@dataclass
class EventStats:
    mean: float
    var: float
    winrate: float
    sharpe: float
    n: int

def to_cupy_no_nulls(gser: cudf.Series, fill_value) -> cp.ndarray:
    return gser.astype("float64").fillna(fill_value).to_cupy()

def _rolling_from_extrema_gser(gser: cudf.Series, years: int, mode: str, minp: int = 50) -> cudf.Series:
    """Distance to multi-year extrema: from_low or from_high."""
    win = years * TRADING_DAYS
    if mode == "from_low":
        trough = gser.rolling(window=win, min_periods=minp).min()
        out = ((gser - trough) / trough).clip(lower=0)
    elif mode == "from_high":
        peak = gser.rolling(window=win, min_periods=minp).max()
        out = ((peak - gser) / peak).clip(lower=0)
    else:
        raise ValueError("mode must be 'from_high' or 'from_low'")
    return out.fillna(0.0)

def _event_stats_gpu(price_cp: cp.ndarray, trigger_mask_cp: cp.ndarray, hold_d: int=EVENT_EVAL_HOLD_D) -> EventStats:
    """Under the trigger mask, compute hold_d-day event returns (GPU)."""
    T = int(price_cp.shape[0])
    if T < hold_d + 2:
        return EventStats(0.0, 0.0, 0.0, 0.0, 0)
    ret = cp.empty(T, dtype=cp.float64); ret[0] = 0.0
    ret[1:] = price_cp[1:] / price_cp[:-1] - 1.0
    L = cp.log1p(ret)
    prefix = cp.concatenate([cp.array([0.0], dtype=cp.float64), cp.cumsum(L)])
    idx = cp.where(trigger_mask_cp.astype(cp.bool_))[0]
    valid = idx[idx + hold_d < T]
    if valid.size == 0:
        return EventStats(0.0, 0.0, 0.0, 0.0, 0)
    r = cp.exp(prefix[valid + hold_d + 1] - prefix[valid + 1]) - 1.0
    mu  = float(cp.mean(r).get())
    if r.size > 1:
        var = float(cp.var(r, ddof=1).get()); std = float(cp.std(r, ddof=1).get())
    else:
        var = float((r[0]**2).get()); std = float(cp.abs(r[0]).get())
    win = float((cp.mean((r > 0).astype(cp.float64))).get())
    shp = 0.0 if std == 0 else (mu / std) * math.sqrt(TRADING_DAYS/hold_d)
    return EventStats(mu, var, win, shp, int(valid.size))

def kelly_from_stats(stats: EventStats) -> float:
    if stats.var <= 1e-12 or math.isnan(stats.var): return 0.0
    return float(stats.mean / stats.var)

def choose_side_gpu(price_pd: pd.Series, years: int, thr: float) -> str:
    """Given a threshold, choose from_low/from_high based on historical stats."""
    gser = cudf.from_pandas(price_pd)
    m_low  = _rolling_from_extrema_gser(gser, years, "from_low")
    m_high = _rolling_from_extrema_gser(gser, years, "from_high")
    price_cp = cp.asarray(price_pd.values, dtype=cp.float64)
    trig_low_cp  = (to_cupy_no_nulls(m_low,  0.0) >= thr)
    trig_high_cp = (to_cupy_no_nulls(m_high, 0.0) >= thr)
    st_low  = _event_stats_gpu(price_cp, trig_low_cp)
    st_high = _event_stats_gpu(price_cp, trig_high_cp)
    return "from_low" if kelly_from_stats(st_low) >= kelly_from_stats(st_high) else "from_high"

def factor_gate(stats: EventStats,
                min_sharpe: float,
                min_win: float,
                min_kelly_raw: float,
                mode: str="all") -> bool:
    """Admission filter for event statistics."""
    conds = [
        (stats.sharpe  >= min_sharpe),
        (stats.winrate >= min_win),
        (kelly_from_stats(stats) >= min_kelly_raw),
    ]
    return all(conds) if mode=="all" else any(conds)

def rolling_std_cp(vec: cp.ndarray, win: int) -> cp.ndarray:
    """Rolling standard deviation on GPU (simplified)."""
    n = vec.shape[0]
    if n < win: return cp.full((n,), cp.nan, dtype=cp.float64)
    wv = cp.lib.stride_tricks.sliding_window_view(vec, win)
    stds = wv.std(axis=1, ddof=0)
    pad = cp.full((win-1,), cp.nan, dtype=cp.float64)
    return cp.concatenate([pad, stds])

# =========================
# 3) Backtest (fixed parameters)
# =========================
@dataclass
class Position:
    is_open: bool = False
    entry_price: float = np.nan
    entry_idx: int = -1
    cooldown_left: int = 0
    min_dd_since_entry: float = 0.0  # deepest drawdown (simple %) since entry

def get_cost_for_symbol(symbol_alias: str) -> Dict[str, float]:
    base = {**GLOBAL_DEFAULT}; base.update(COST_TABLE.get(symbol_alias, {}))
    return dict(
        SLIPPAGE_BPS=float(base.get("SLIPPAGE_BPS", 0.0)),
        COMMISSION_BPS=float(base.get("COMMISSION_BPS", 0.0)),
        STAMP_DUTY_BPS=float(base.get("STAMP_DUTY_BPS", 0.0)),
    )

def tc_buy_effect_bps(symbol_alias: str) -> float:
    c = get_cost_for_symbol(symbol_alias)
    return c["SLIPPAGE_BPS"] + c["COMMISSION_BPS"]

def tc_sell_effect_bps(symbol_alias: str) -> float:
    c = get_cost_for_symbol(symbol_alias)
    return c["SLIPPAGE_BPS"] + c["COMMISSION_BPS"] + c["STAMP_DUTY_BPS"]

def backtest_once(prices: pd.DataFrame,
                  used_alias_map: Dict[str,str],
                  cooldown_d: int = 0,
                  default_band_thresh: float = DEFAULT_BAND_THRESH) -> Tuple[pd.Series, pd.DataFrame]:
    tickers = list(prices.columns)
    dates = prices.index
    T, N = prices.shape

    gpx = cudf.from_pandas(prices)
    dd_or_up_np = np.zeros((T, N), dtype=np.float64)
    ma_np       = np.full((T, N), np.nan, dtype=np.float64)
    slope_ok_np = np.zeros((T, N), dtype=bool)
    ev_stats: Dict[str, EventStats] = {}
    gates_ok: Dict[str, bool] = {}

    # Precompute side selection, trigger metric, and trend filters
    for j, tk in enumerate(tickers):
        thr = BAND_THRESH.get(tk, default_band_thresh)
        side = choose_side_gpu(prices[tk], LOOKBACK_YEARS, thr)

        metric_g = _rolling_from_extrema_gser(gpx[tk], LOOKBACK_YEARS, side)
        dd_or_up_np[:, j] = cp.asnumpy(to_cupy_no_nulls(metric_g, 0.0))

        ma_g = gpx[tk].rolling(window=MA_LONG, min_periods=MA_LONG//2).mean().fillna(np.nan)
        ma_np[:, j] = cp.asnumpy(to_cupy_no_nulls(ma_g, np.nan))
        slope_ok_np[:, j] = cp.asnumpy((ma_g.diff(MA_SLOPE_WIN) > 0).fillna(False).to_cupy()) if MA_SLOPE_FILTER_ON else True

        price_cp = cp.asarray(prices[tk].values, dtype=cp.float64)
        trig_cp  = (to_cupy_no_nulls(metric_g, 0.0) >= thr)
        st = _event_stats_gpu(price_cp, trig_cp, hold_d=EVENT_EVAL_HOLD_D)
        ev_stats[tk] = st
        gates_ok[tk] = (st.n >= MIN_EVENTS) and factor_gate(
            st, FACTOR_MIN_EVENT_SHARPE, FACTOR_MIN_WINRATE, FACTOR_MIN_KELLY_RAW, FACTOR_GATE_MODE
        )

    # Daily returns and vol
    px_cp  = cp.asarray(prices.values, dtype=cp.float64)
    rets_cp = cp.zeros((T, N), dtype=cp.float64)
    rets_cp[1:, :] = px_cp[1:, :] / px_cp[:-1, :] - 1.0
    cp.nan_to_num(rets_cp, copy=False, nan=0.0, posinf=0.0, neginf=0.0)
    rets_np = cp.asnumpy(rets_cp)

    sigma20_np = np.empty((T, N), dtype=np.float64)
    for j in range(N):
        sigma20_np[:, j] = cp.asnumpy(rolling_std_cp(rets_cp[:, j], 20))

    # Position states
    pos = {tk: Position() for tk in tickers}
    w_base_np = np.zeros((T, N), dtype=np.float64)
    dd_close_cols = np.full((T, N), np.nan, dtype=np.float64)  # record the trade's deepest DD on exit day

    for t in range(T):
        # 1) Compute "raw" Kelly-based weight quota
        k_used = np.zeros(N, dtype=np.float64)
        for j, tk in enumerate(tickers):
            thr = BAND_THRESH.get(tk, default_band_thresh)
            trig_today = (dd_or_up_np[t, j] >= thr)
            gate_ok = gates_ok[tk]
            if LONG_TREND_FILTER_ON:
                ma_v = ma_np[t, j]
                trend_ok = (not np.isnan(ma_v)) and (prices.iloc[t, j] >= ma_v) and bool(slope_ok_np[t, j])
            else:
                trend_ok = True
            if trig_today and gate_ok and trend_ok and pos[tk].cooldown_left == 0:
                kr = kelly_from_stats(ev_stats[tk])
                ku = max(KELLY_MIN_USED, min(kr*KELLY_FRACTION, CAP_PER_ASSET_LONG))
            else:
                ku = 0.0
            k_used[j] = ku

        # 2) Top-K normalization (cap number of names per day)
        if TOPK_PER_DAY and TOPK_PER_DAY > 0:
            idx_pos = np.where(k_used > 0)[0]
            idx_sorted = idx_pos[np.argsort(k_used[idx_pos])[::-1]]
            keep = idx_sorted[:TOPK_PER_DAY]
        else:
            keep = np.where(k_used > 0)[0]

        total_k = k_used[keep].sum() if keep.size > 0 else 0.0
        base_w = np.zeros(N, dtype=np.float64)
        if total_k > 0: base_w[keep] = k_used[keep] / total_k

        # 3) Check stops/take-profit/time-stop for open positions
        for j, tk in enumerate(tickers):
            if pos[tk].is_open:
                start_loc = pos[tk].entry_idx
                if (start_loc is None) or (not isinstance(start_loc, (int, np.integer))) or (start_loc < 0) or (start_loc >= t):
                    pos[tk] = Position(is_open=False, cooldown_left=cooldown_d); continue
                r_cum = (1.0 + rets_np[start_loc+1:t+1, j]).prod() - 1.0 if t > start_loc else 0.0
                pos[tk].min_dd_since_entry = min(pos[tk].min_dd_since_entry, r_cum)
                vol = sigma20_np[t, j]
                stop_hit_sigma = (not np.isnan(vol)) and (vol > 0) and (r_cum <= -STOP_SIGMA * vol)
                take_hit       = (not np.isnan(vol)) and (vol > 0) and (r_cum >=  TAKE_SIGMA * vol)
                time_stop      = (TIME_STOP_D is not None) and ((t - start_loc) >= TIME_STOP_D)
                stop_hit_entry = ENTRY_DD_STOP_ON and (r_cum <= -ENTRY_DD_STOP_PCT)
                if stop_hit_entry or stop_hit_sigma or take_hit or time_stop:
                    dd_close_cols[t, j] = pos[tk].min_dd_since_entry
                    pos[tk] = Position(is_open=False, cooldown_left=cooldown_d)

        # 4) Target weights (open/maintain)
        tw = np.zeros(N, dtype=np.float64)
        for j, tk in enumerate(tickers):
            if pos[tk].cooldown_left > 0:
                tw[j] = 0.0
            else:
                px_ = prices.iloc[t, j]
                if (not pos[tk].is_open) and base_w[j] > 0 and (not np.isnan(px_)):
                    pos[tk].is_open = True
                    pos[tk].entry_price = px_
                    pos[tk].entry_idx = t
                    pos[tk].min_dd_since_entry = 0.0
                    tw[j] = base_w[j]
                elif pos[tk].is_open:
                    tw[j] = base_w[j]
                else:
                    tw[j] = 0.0
        w_base_np[t, :] = tw

        # 5) Cooldown decrement
        for tk in tickers:
            if pos[tk].cooldown_left > 0:
                pos[tk].cooldown_left -= 1

    # Execution lag (T+1)
    w_exec_np = np.vstack([np.zeros((TRADE_LAG_D, N)), w_base_np[:-TRADE_LAG_D, :]]) if TRADE_LAG_D > 0 else w_base_np.copy()

    # ---- Portfolio NAV execution and transaction costs ----
    nav_cp = cp.ones(T, dtype=cp.float64)
    cur_w_cp = cp.zeros(N, dtype=cp.float64)

    buy_bps  = np.array([tc_buy_effect_bps(used_alias_map.get(tk, tk)) for tk in tickers], dtype=np.float64)
    sell_bps = np.array([tc_sell_effect_bps(used_alias_map.get(tk, tk)) for tk in tickers], dtype=np.float64)
    buy_rt  = cp.asarray(-np.abs(buy_bps)/1e4, dtype=cp.float64)
    sell_rt = cp.asarray(-np.abs(sell_bps)/1e4, dtype=cp.float64)

    # Only rebalance on the specified frequency (default Fridays)
    freq_mask = np.ones(T, dtype=bool) if VOL_REBAL_FREQ == "D" else (pd.to_datetime(dates).weekday == 4).astype(bool)

    for i in range(T):
        if i > 0:
            day_ret = float((cur_w_cp @ rets_cp[i, :]).get())
            nav_cp[i] = nav_cp[i-1] * (1.0 + day_ret)
        if not freq_mask[i]:
            continue

        eff_target = TARGET_DAILY_VOL
        if DRAWDOWN_BRAKE_ON:
            start_win = max(0, i - BRAKE_WINDOW_D)
            peak = float(cp.max(nav_cp[start_win:i+1]).get())
            if peak > 0:
                dd_now = float((nav_cp[i] / peak - 1.0).get())
                if dd_now <= -BRAKE_THRESH:
                    eff_target = TARGET_DAILY_VOL * BRAKE_SCALE

        tgt_np = w_exec_np[i, :]
        if tgt_np.sum() > 0:
            start_cov = max(0, i - VOL_LOOKBACK_D)
            window = rets_cp[start_cov:i+1, :]
            window = cp.nan_to_num(window, copy=False, nan=0.0, posinf=0.0, neginf=0.0)
            if window.shape[0] >= 2:
                cov = cp.cov(window, rowvar=False)
                cur = cp.asarray(tgt_np, dtype=cp.float64)
                port_sig = cp.sqrt(cur @ cov @ cur)
                if float(port_sig.get()) > 0:
                    cur = cur * (eff_target / port_sig)
                gross = cp.sum(cp.abs(cur))
                if float(gross.get()) > GROSS_LEV_MAX:
                    cur = cur * (GROSS_LEV_MAX / gross)

                if i > 0:
                    delta = cur - cur_w_cp
                    buy_turn  = cp.clip(delta, 0, None)
                    sell_turn = cp.clip(-delta, 0, None)
                    nav_cp[i] = nav_cp[i] * (1.0 + float((buy_turn @ buy_rt).get()) + float((sell_turn @ sell_rt).get()))
                cur_w_cp = cur
            else:
                cur_w_cp = cp.asarray(tgt_np, dtype=cp.float64)
        else:
            if i > 0:
                delta = -cur_w_cp
                sell_turn = cp.clip(-delta, 0, None)
                nav_cp[i] = nav_cp[i] * (1.0 + float((sell_turn @ sell_rt).get()))
            cur_w_cp = cp.zeros(N, dtype=cp.float64)

    nav_np = cp.asnumpy(nav_cp)
    detail = pd.DataFrame(index=dates)
    detail["NAV"] = nav_np
    for j, tk in enumerate(tickers):
        detail[f"W_base_{tk}"] = w_base_np[:, j]
        detail[f"W_exec_{tk}"] = w_exec_np[:, j]
        detail[f"DDclose_{tk}"] = dd_close_cols[:, j]

    return detail["NAV"], detail

# =========================
# 4) Dashboard & utility functions
# =========================
def max_drawdown_from_entry_simple(detail: pd.DataFrame) -> float:
    """Portfolio-level minimum of per-trade drawdowns booked on exit day (simple % from entry)."""
    cols = [c for c in detail.columns if c.startswith("DDclose_")]
    if not cols:
        return 0.0
    vals = pd.concat([detail[c] for c in cols], axis=0).dropna()
    if vals.empty:
        return 0.0
    return float(vals.min())

def extrema_high_low_table(prices: pd.DataFrame, lookbacks=(5,10,20)) -> pd.DataFrame:
    """Table of highs/lows over 5/10/20-year lookbacks."""
    rows = []
    last = prices.index[-1]
    for tk in prices.columns:
        s = prices[tk].dropna()
        rec = {"Ticker": tk}
        for y in lookbacks:
            start = last - pd.tseries.offsets.BDay(int(y*TRADING_DAYS))
            s_win = s[s.index >= start]
            if len(s_win) == 0:
                rec.update({f"Low_{y}Y": np.nan, f"LowDate_{y}Y": np.nan,
                            f"High_{y}Y": np.nan, f"HighDate_{y}Y": np.nan})
            else:
                lo = float(s_win.min()); hi = float(s_win.max())
                lo_dt = s_win.idxmin(); hi_dt = s_win.idxmax()
                rec.update({f"Low_{y}Y": round(lo, 4), f"LowDate_{y}Y": lo_dt.date(),
                            f"High_{y}Y": round(hi,4), f"HighDate_{y}Y": hi_dt.date()})
        rows.append(rec)
    df = pd.DataFrame(rows)
    cols = ["Ticker"]
    for y in lookbacks:
        cols += [f"Low_{y}Y", f"LowDate_{y}Y", f"High_{y}Y", f"HighDate_{y}Y"]
    return df[cols]

def conditional_event_table(prices: pd.DataFrame,
                            thresholds=(0.02,0.05,0.08),
                            lookbacks=(5,10,20),
                            hold_d: int = EVENT_EVAL_HOLD_D) -> pd.DataFrame:
    """Conditional event stats table (from_low/from_high × threshold × holding period)."""
    rows = []
    gpx = cudf.from_pandas(prices)
    for tk in prices.columns:
        px_pd = prices[tk]
        px_cp = cp.asarray(px_pd.values, dtype=cp.float64)
        for lb in lookbacks:
            for mode in ("from_low","from_high"):
                metric_g = _rolling_from_extrema_gser(gpx[tk], lb, mode)
                for thr in thresholds:
                    trig_cp  = (to_cupy_no_nulls(metric_g, 0.0) >= thr)
                    st = _event_stats_gpu(px_cp, trig_cp, hold_d=hold_d)
                    rows.append({
                        "Ticker": tk, "LookbackY": lb, "Mode": mode, "Threshold": thr,
                        "WinRate": round(st.winrate, 3),
                        "EventSharpe": round(st.sharpe, 3),
                        "Mean": round(st.mean, 6),
                        "Var": round(st.var, 8),
                        "Kelly_raw": round(kelly_from_stats(st), 4),
                        "N_events": st.n
                    })
    df = pd.DataFrame(rows)
    return df.sort_values(["Ticker","Mode","LookbackY","Threshold"]).reset_index(drop=True)

def today_signal_table(prices: pd.DataFrame,
                       used_map: Dict[str,str],
                       as_of: Optional[pd.Timestamp]=None,
                       default_band_thresh: float = DEFAULT_BAND_THRESH) -> pd.DataFrame:
    """Today's signal table: triggers/gates, Kelly_raw, and suggested weights (after TopK)."""
    if as_of is None:
        as_of = prices.index[-1]
    as_of = pd.to_datetime(as_of)
    if as_of not in prices.index:
        idx_asof = prices.index.searchsorted(as_of, side='right') - 1
        idx_asof = max(0, min(idx_asof, len(prices.index)-1))
        as_of = prices.index[idx_asof]

    gpx = cudf.from_pandas(prices)
    rows = []
    k_used_map = {}

    for tk in prices.columns:
        thr = BAND_THRESH.get(tk, default_band_thresh)
        side = choose_side_gpu(prices[tk], LOOKBACK_YEARS, thr)
        metric_g = _rolling_from_extrema_gser(gpx[tk], LOOKBACK_YEARS, side)

        # Align index
        idx_m = metric_g.index.searchsorted(as_of, side='right') - 1
        idx_m = max(0, min(int(idx_m), len(metric_g)-1))
        dd_or_up = float(metric_g.iloc[idx_m])

        # Event stats (gating + kelly_raw)
        price_cp = cp.asarray(prices[tk].values, dtype=cp.float64)
        trig_cp  = (to_cupy_no_nulls(metric_g, 0.0) >= thr)
        st = _event_stats_gpu(price_cp, trig_cp, hold_d=EVENT_EVAL_HOLD_D)
        gate_ok = (st.n >= MIN_EVENTS) and factor_gate(st, FACTOR_MIN_EVENT_SHARPE, FACTOR_MIN_WINRATE, FACTOR_MIN_KELLY_RAW, FACTOR_GATE_MODE)

        # Trend filter
        ma_g = gpx[tk].rolling(window=MA_LONG, min_periods=MA_LONG//2).mean().fillna(np.nan)
        idx_ma = ma_g.index.searchsorted(as_of, side='right') - 1
        idx_ma = max(0, min(int(idx_ma), len(ma_g)-1))
        ma_val = float(ma_g.iloc[idx_ma])
        price_today = float(prices.loc[as_of, tk])
        slope_series = (ma_g.diff(MA_SLOPE_WIN) > 0).fillna(False)
        slope_ok = bool(slope_series.iloc[idx_ma]) if MA_SLOPE_FILTER_ON else True
        trend_ok = (not np.isnan(ma_val)) and (price_today >= ma_val) and slope_ok

        trig_today = (dd_or_up >= thr)
        kr = kelly_from_stats(st)
        ku = max(KELLY_MIN_USED, min(kr*KELLY_FRACTION, CAP_PER_ASSET_LONG)) if (trig_today and gate_ok and trend_ok) else 0.0
        k_used_map[tk] = ku

        rows.append({
            "Ticker": tk,
            "AliasUsed": used_map.get(tk, tk),
            "Side": side,
            "Trigger(>=th)": round(dd_or_up, 4),
            "Threshold": thr,
            "GateOK": gate_ok,
            "TrendOK": trend_ok,
            "Kelly_raw": round(kr, 4),
            "k_used_raw": round(ku, 4),
        })

    # TopK normalization → final suggested weights
    arr = np.array([k_used_map[tk] for tk in prices.columns], dtype=float)
    idx_pos = np.where(arr > 0)[0]
    idx_sorted = idx_pos[np.argsort(arr[idx_pos])[::-1]]
    keep = idx_sorted[:TOPK_PER_DAY]
    final_w = np.zeros_like(arr)
    if keep.size > 0:
        s = arr[keep].sum()
        if s > 0:
            final_w[keep] = arr[keep] / s

    # Write-back of weights and decision suggestions
    for r in rows:
        tk = r["Ticker"]
        w = float(final_w[list(prices.columns).index(tk)])
        r["SuggestedWeight"] = round(w, 4)
        r["Decision"] = ("OPEN/ADD" if w > 0 else "STAY OUT")

    df = pd.DataFrame(rows)
    df = df.sort_values(["SuggestedWeight","k_used_raw"], ascending=[False, False]).reset_index(drop=True)
    return df

def evaluate_once(prices: pd.DataFrame, used_map: Dict[str,str], cooldown: int = 0,
                  default_band_thresh: float = DEFAULT_BAND_THRESH) -> Tuple[pd.DataFrame, Dict[int, pd.DataFrame]]:
    """Full-history backtest (fixed parameters)."""
    nav, det = backtest_once(prices, used_map, cooldown_d=cooldown, default_band_thresh=default_band_thresh)
    total_ret = float(nav.iloc[-1] - 1)
    daily_ret = nav.pct_change().fillna(0.0)
    vol = float(daily_ret.std())
    sharpe = (daily_ret.mean()/vol)*math.sqrt(TRADING_DAYS) if vol>0 else 0.0
    mdd_simple = max_drawdown_from_entry_simple(det)
    years = max(1e-9, len(nav)/TRADING_DAYS)
    cagr = float(nav.iloc[-1])**(1/years) - 1
    calmar = (cagr/abs(mdd_simple)) if mdd_simple < 0 else np.nan
    R_over_MDD = (total_ret/abs(mdd_simple)) if mdd_simple != 0 else float('inf')

    summ_row = {
        "CooldownD": int(cooldown),
        "TotalReturn%": round(total_ret*100, 2),
        "CAGR%": round(cagr*100, 2),
        "Ann.Vol%": round(annualize_vol(vol)*100, 2),
        "Sharpe": round(sharpe, 3),
        "MaxDD%": round(mdd_simple*100, 2),
        "R_over_MDD": round(R_over_MDD, 2) if np.isfinite(R_over_MDD) else "∞",
        "Calmar": round(calmar, 3) if not np.isnan(calmar) else np.nan
    }
    return pd.DataFrame([summ_row]), {int(cooldown): det}

def summarize_period(nav: pd.Series, detail: pd.DataFrame,
                     start_date: str, end_date: Optional[str]=None) -> Tuple[pd.DataFrame, pd.Series]:
    """Period summary (includes period MaxDD under this strategy's definition)."""
    d0 = pd.to_datetime(start_date)
    d1 = pd.to_datetime(end_date) if end_date is not None else nav.index[-1]
    sub = nav[(nav.index >= d0) & (nav.index <= d1)]
    if sub.empty:
        raise ValueError("No data in the selected period.")
    nav0 = float(sub.iloc[0])
    navN = sub / nav0
    daily = navN.pct_change().fillna(0.0)
    vol = float(daily.std())
    years = max(1e-9, len(sub)/TRADING_DAYS)
    cagr = float(navN.iloc[-1])**(1/years) - 1

    # Use only exit records within the period to compute MaxDD
    m = detail[(detail.index >= d0) & (detail.index <= d1)]
    cols = [c for c in m.columns if c.startswith("DDclose_")]
    vals = pd.concat([m[c] for c in cols], axis=0).dropna()
    mdd_simple = float(vals.min()) if not vals.empty else 0.0

    total_ret = float(navN.iloc[-1] - 1)
    sharpe = (daily.mean()/vol)*math.sqrt(TRADING_DAYS) if vol>0 else 0.0
    calmar = (cagr/abs(mdd_simple)) if mdd_simple < 0 else np.nan
    R_over_MDD = (total_ret/abs(mdd_simple)) if mdd_simple != 0 else float('inf')

    summ = pd.DataFrame([{
        "Start": d0.date(), "End": d1.date(),
        "TotalReturn%": round(total_ret*100, 2),
        "CAGR%": round(cagr*100, 2),
        "Ann.Vol%": round(annualize_vol(vol)*100, 2),
        "Sharpe": round(sharpe, 3),
        "MaxDD%": round(mdd_simple*100, 2),
        "R_over_MDD": round(R_over_MDD, 2) if np.isfinite(R_over_MDD) else "∞",
        "Calmar": round(calmar, 3) if not np.isnan(calmar) else np.nan
    }])
    return summ, navN

def build_trade_schedule(detail: pd.DataFrame,
                         start_date: Optional[str]=None,
                         end_date: Optional[str]=None,
                         change_tol: float = 1e-4) -> pd.DataFrame:
    """Extract actual order-weight changes from W_exec_* columns to form a rebalance schedule."""
    df = detail.copy()
    if start_date is not None:
        df = df[df.index >= pd.to_datetime(start_date)]
    if end_date is not None:
        df = df[df.index <= pd.to_datetime(end_date)]

    wcols = [c for c in df.columns if c.startswith("W_exec_")]
    tickers = [c.replace("W_exec_", "") for c in wcols]
    W = df[wcols].fillna(0.0)

    # Identify rows with significant weight changes (including the first day)
    changed = [0]
    prev = W.iloc[0].values
    for i in range(1, len(W)):
        cur = W.iloc[i].values
        if np.max(np.abs(cur - prev)) > change_tol:
            changed.append(i)
            prev = cur
    out = df.iloc[changed][wcols].copy()
    out.columns = tickers

    # Attach natural-language actions (Open/Increase/Decrease/Close)
    actions = []
    prev = None
    for _, row in out.iterrows():
        cur = row.values
        if prev is None:
            prev = np.zeros_like(cur)
        msgs = []
        for j, tk in enumerate(tickers):
            a, b = prev[j], cur[j]
            if a < change_tol and b >= change_tol:
                msgs.append(f"{tk}: OPEN→{b:.2%}")
            elif a >= change_tol and b < change_tol:
                msgs.append(f"{tk}: CLOSE")
            elif abs(b - a) >= 1e-3:
                msgs.append(f"{tk}: {('ADD' if b>a else 'CUT')} {abs(b-a):.2%}")
        actions.append("; ".join(msgs))
        prev = cur
    out.insert(0, "Actions", actions)
    out.index.name = "Date"
    return out.reset_index()

# =========================
# 5) Report export (Institutional edition)
# =========================
def export_institutional_report(prices: pd.DataFrame, used_map: Dict[str,str],
                                scenario_start: Optional[str] = None,
                                scenario_end: Optional[str] = None,
                                outfile: str = "kelly_institutional_report.xlsx"):
    # 1) Full-history backtest (fixed cooldown=0)
    summ_full, details_dict = evaluate_once(prices, used_map, cooldown=0, default_band_thresh=DEFAULT_BAND_THRESH)
    det_full = details_dict[0]

    # 2) Dashboard sheets
    hilo = extrema_high_low_table(prices, lookbacks=(5,10,20))
    prob = conditional_event_table(prices, thresholds=(0.02,0.05,0.08), lookbacks=(5,10,20), hold_d=EVENT_EVAL_HOLD_D)

    # 3) Today's signals
    today_tbl = today_signal_table(prices, used_map, as_of=None, default_band_thresh=DEFAULT_BAND_THRESH)

    # 4) Scenario A: from scenario_start to latest (or scenario_end)
    if scenario_start is None:
        # Default to last one year: roll back 252 business days from the last trading date
        last_dt = det_full.index[-1]
        scenario_start = (last_dt - pd.tseries.offsets.BDay(TRADING_DAYS)).date().isoformat()

    summ_a, nav_a = summarize_period(det_full["NAV"], det_full, start_date=scenario_start, end_date=scenario_end)
    sched_a = build_trade_schedule(det_full, start_date=scenario_start, end_date=scenario_end)

    # 5) Scenario B: full-history schedule
    sched_full = build_trade_schedule(det_full, start_date=None, end_date=None)

    # 6) Performance bar check
    gate = pd.DataFrame([{
        "Target_CAGR%": 10.0,
        "Target_R_over_MDD": 3.0,
        "Achieved_CAGR%": float(summ_full.loc[0, "CAGR%"]),
        "Achieved_R_over_MDD": summ_full.loc[0, "R_over_MDD"],
        "Pass?": (float(summ_full.loc[0, "CAGR%"])>=10.0) and ( (summ_full.loc[0, "R_over_MDD"]=="∞") or (float(summ_full.loc[0, "R_over_MDD"])>=3.0) )
    }])

    # 7) Write Excel
    with pd.ExcelWriter(outfile, engine="openpyxl") as ew:
        # Parameters sheet
        meta = pd.Series({
            "PREFERRED_MODE": PREFERRED_MODE,
            "ALLOW_FALLBACK_TO_ETF": ALLOW_FALLBACK_TO_ETF,
            "START": START, "END": END, "TRADING_DAYS": TRADING_DAYS,
            "LOOKBACK_YEARS": LOOKBACK_YEARS, "MA_LONG": MA_LONG,
            "EVENT_EVAL_HOLD_D": EVENT_EVAL_HOLD_D, "MIN_EVENTS": MIN_EVENTS,
            "KELLY_FRACTION": KELLY_FRACTION, "CAP_PER_ASSET_LONG": CAP_PER_ASSET_LONG,
            "TOPK_PER_DAY": TOPK_PER_DAY,
            "TARGET_DAILY_VOL": TARGET_DAILY_VOL, "VOL_LOOKBACK_D": VOL_LOOKBACK_D,
            "VOL_REBAL_FREQ": VOL_REBAL_FREQ, "GROSS_LEV_MAX": GROSS_LEV_MAX,
            "STOP_SIGMA": STOP_SIGMA, "TAKE_SIGMA": TAKE_SIGMA,
            "ENTRY_DD_STOP_ON": ENTRY_DD_STOP_ON, "ENTRY_DD_STOP_PCT": ENTRY_DD_STOP_PCT,
            "TIME_STOP_D": TIME_STOP_D,
            "BRAKE_THRESH": BRAKE_THRESH, "BRAKE_SCALE": BRAKE_SCALE,
            "MaxDD_Definition": "Per-trade drawdown from entry (simple %); portfolio MaxDD is the minimum across trades on close",
            "CANONICAL_SYMBOLS_USED": str(list(prices.columns)),
            "USED_ALIASES": str(used_map),
        }).to_frame("value")
        meta.index.name = "Param"
        meta.to_excel(ew, sheet_name="Params")

        summ_full.to_excel(ew, sheet_name="Summary_Full", index=False)
        gate.to_excel(ew, sheet_name="Gate_Check", index=False)
        det_full.to_excel(ew, sheet_name="Detail_Full")

        hilo.to_excel(ew, sheet_name="Extrema_5_10_20Y", index=False)
        prob.to_excel(ew, sheet_name="Event_Prob_2-5-8", index=False)
        today_tbl.to_excel(ew, sheet_name="Today_Signals", index=False)

        summ_a.to_excel(ew, sheet_name="Scenario_A_Summary", index=False)
        nav_a.rename("NAV_A").to_frame().to_excel(ew, sheet_name="Scenario_A_NAV")
        sched_a.to_excel(ew, sheet_name="Scenario_A_Schedule", index=False)

        sched_full.to_excel(ew, sheet_name="Schedule_Full", index=False)

    print("✅ Report generated:", outfile)
    print("- Full-history performance:\n", summ_full.to_string(index=False))
    print("- Scenario A performance:\n", summ_a.to_string(index=False))
    print("- Today's signals (post-TopK):\n", today_tbl.head(10).to_string(index=False))

# =========================
# 6) Main flow (default: Scenario A = last one year)
# =========================
def run_institutional_report():
    print("\n=== Loading asset universe (FULL) ===")
    prices, used = load_prices_robust(CANONICAL_SYMBOLS_FULL, START, END)
    print("[info] Aliases used:", used)

    print("\n=== Generating institutional report ===")
    export_institutional_report(
        prices, used,
        scenario_start=None,      # None = last 1y; or specify e.g. "2025-08-01"
        scenario_end=None,        # None = latest; or specify e.g. "2025-09-22"
        outfile="kelly_institutional_report.xlsx"
    )

if __name__ == "__main__":
    run_institutional_report()



=== Loading asset universe (FULL) ===


ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: US_SPX"}}}
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['US_SPX']: YFTzMissingError('possibly delisted; no timezone found')
ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: US_SPX"}}}
ERROR:yfinance:$US_SPX: possibly delisted; no timezone found
[try] US_SPX → US_SPX ... no data
[warn] yf.download failed for ES=F: 'str' object is not callable


[ok] US_SPX → ES=F (5222 rows)


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['US_NASDAQ']: YFTzMissingError('possibly delisted; no timezone found')
ERROR:yfinance:$US_NASDAQ: possibly delisted; no timezone found
[try] US_NASDAQ → US_NASDAQ ... no data
[warn] yf.download failed for NQ=F: 'str' object is not callable


[ok] US_NASDAQ → NQ=F (5222 rows)


...

[ok] 700HK → 0700.HK (5112 rows)


[warn] yf.download failed for 0027.HK: 'str' object is not callable


[ok] 27HK → 0027.HK (5112 rows)


[warn] yf.download failed for 0883.HK: 'str' object is not callable


[ok] 883HK → 0883.HK (5112 rows)
[info] Aliases used: {'US_SPX': 'ES=F', 'US_NASDAQ': 'NQ=F', 'SPY': 'SPY', 'QQQ': 'QQQ', 'FEZ': 'FEZ', 'EWJ': 'EWJ', 'FXI': 'FXI', 'GLD': 'GLD', 'SLV': 'SLV', 'AAPL': 'AAPL', 'TSLA': 'TSLA', 'MAR': 'MAR', 'BABA': 'BABA', 'FSLR': 'FSLR', 'SBUX': 'SBUX', 'PLTR': 'PLTR', 'NVDA': 'NVDA', '700HK': '0700.HK', '27HK': '0027.HK', '883HK': '0883.HK'}

=== Generating institutional report ===
✅ Report generated: kelly_institutional_report.xlsx
- Full-history performance:
  CooldownD  TotalReturn%  CAGR%  Ann.Vol%  Sharpe  MaxDD%  R_over_MDD  Calmar
         0       3019.54  17.61     13.88   1.238  -12.39      243.62   1.421
- Scenario A performance:
      Start        End  TotalReturn%  CAGR%  Ann.Vol%  Sharpe  MaxDD%  R_over_MDD  Calmar
2024-10-04 2025-09-23          8.94   9.01     12.99   0.729  -12.39        0.72   0.727
- Today's signals (post-TopK):
  Ticker AliasUsed      Side  Trigger(>=th)  Threshold  GateOK  TrendOK  Kelly_raw  k_used_raw  SuggestedWeig